In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pickle
import time

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16308016401801025340
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1977404622
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3073456010993102133
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:07:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [3]:
# Settings
num_epochs = 10
batch_size = 32
dense_layers = [0, 1, 2] # [0, 1, 2] [0]
layer_sizes = [64] # [64] [32, 64, 128]
conv_layers = [3] # [3] [1,2,3]
activation_functions = ['relu', 'sigmoid']
optimiser = ['adam']



In [4]:
# gpu_options = tf.config.gpu.set_per_process_memory_fraction(FRAC)
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sessiom = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [5]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)
y = np.array(y)
X = X/255.0

In [7]:
for dense_layer in dense_layers:
    print(f"----Dense layer: {dense_layer}----")
    for layer_size in layer_sizes:
        print(f"----layer size: {layer_size}----")
        for conv_layer in conv_layers:
            print(f"----Conv layer: {conv_layer}----")
            NAME = f"d--{batch_size}-batch-{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            print(NAME)
            
            model = Sequential()
            
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(512)) 
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

            model.fit(X,y, batch_size=batch_size, epochs=num_epochs, validation_split=0.3, callbacks=[tensorboard])

----Dense layer: 0----
----layer size: 64----
----Conv layer: 3----
c--32-batch-3-conv-64-nodes-0-dense-1657072217
Epoch 1/20
546/546 [==============================] - 9s 15ms/step - loss: 0.6935 - accuracy: 0.5612 - val_loss: 0.6316 - val_accuracy: 0.6386
Epoch 2/20
546/546 [==============================] - 7s 14ms/step - loss: 0.6350 - accuracy: 0.6507 - val_loss: 0.6625 - val_accuracy: 0.5747
Epoch 3/20
546/546 [==============================] - 7s 13ms/step - loss: 0.6104 - accuracy: 0.6779 - val_loss: 0.5737 - val_accuracy: 0.7104
Epoch 4/20
546/546 [==============================] - 7s 13ms/step - loss: 0.6045 - accuracy: 0.6863 - val_loss: 0.5772 - val_accuracy: 0.7243
Epoch 5/20
546/546 [==============================] - 7s 13ms/step - loss: 0.6081 - accuracy: 0.6902 - val_loss: 0.6371 - val_accuracy: 0.6883
Epoch 6/20
546/546 [==============================] - 7s 13ms/step - loss: 6.2345 - accuracy: 0.5431 - val_loss: 7.5941 - val_accuracy: 0.5020
Epoch 7/20
546/546 [=======